In [14]:
from os.path import join
import bnr.azrael as azrael

import pandas as pd
import numpy as np
import sidetable

In [2]:
data_folder = "data"
az2a = azrael.Azrael2analysis()
az2a.create_az(path_az=join(data_folder, "bnr_d001_20231218.csv.gz"))

az2a.split_path()
az2a.get_extension_mimetype()
az = az2a.az
az['nb'] = 1
az = az[az['path0'].isin(['BNR_VERIF', 'BNR_TAMPON', 'BNR_SAUV'])]
az.columns

Index(['name', 'path', 'md5', 'size', 'last_content_modification_date',
       'last_metadata_modification_date', 'path0', 'path1', 'path2', 'path3',
       'path4', 'extension', 'mimetype', 'guessed_extension', 'nb'],
      dtype='object')

In [64]:
az.stb.missing()

,missing,total,percent
path4,639663,2301003,27.799312
path3,251200,2301003,10.916978
path2,5818,2301003,0.252846
mimetype,5096,2301003,0.221469
guessed_extension,5096,2301003,0.221469
path1,5,2301003,0.000217
name,0,2301003,0.000000
path,0,2301003,0.000000
md5,0,2301003,0.000000
size,0,2301003,0.000000


In [65]:
az.stb.freq(['path0'])

,path0,count,percent,cumulative_count,cumulative_percent
0,BNR_VERIF,1785153,77.581516,1785153,77.581516
1,BNR_SAUV,512461,22.271201,2297614,99.852716
2,BNR_TAMPON,3389,0.147284,2301003,100.000000


In [3]:
az.stb.freq(['path1'])

,path1,count,percent,cumulative_count,cumulative_percent
0,MED,1473739,64.047818,1473739,64.047818
1,AMR,792236,34.430104,2265975,98.477921
2,PAR,14582,0.633725,2280557,99.111646
3,MAGASINES,9487,0.412299,2290044,99.523946
4,LAR,3827,0.166319,2293871,99.690265
5,MUS,2340,0.101695,2296211,99.791960
6,OBS,1802,0.078314,2298013,99.870274
7,LAI,1666,0.072403,2299679,99.942677
8,VAH,458,0.019904,2300137,99.962581
9,CSV,243,0.010561,2300380,99.973142


In [68]:
az.stb.freq(['mimetype'])

,mimetype,count,percent,cumulative_count,cumulative_percent
0,image/jpeg,609032,26.526858,609032,26.526858
1,image/tiff,591680,25.771079,1200712,52.297937
2,application/xml,414639,18.059921,1615351,70.357859
3,application/pdf,337205,14.687224,1952556,85.045082
4,text/plain,334134,14.553464,2286690,99.598546
5,audio/x-wav,8282,0.360729,2294972,99.959275
6,audio/mpeg,630,0.027440,2295602,99.986715
7,video/quicktime,124,0.005401,2295726,99.992116
8,model/vnd.mts,38,0.001655,2295764,99.993772
9,image/x-xcf,38,0.001655,2295802,99.995427


In [30]:
main_gr = az.groupby(['path0', 'path1', 'path2', 'mimetype']).agg(
        {
            'nb': 'sum',
            'size': 'sum'
        }
    ).stb.subtotal(
        sub_level=None,
        grand_label='Total',
        sub_label='Sous-total',
        show_sep=True,
        sep=' | ').reset_index()
main_gr['size_mo'] = main_gr['size'].apply(lambda x: azrael.convert_size(x, to_size='mo'))
main_gr['size_go'] = main_gr['size'].apply(lambda x: azrael.convert_size(x, to_size='go'))
main_gr['size_to'] = main_gr['size'].apply(lambda x: azrael.convert_size(x, to_size='to'))
main_gr.columns = ['path0', 'path1', 'path2', 'mimetype', 'nb', 'size', 'size_mo',
       'size_go', 'size_to']

In [31]:
level0_gr_median = az.groupby(['path0']).agg(
        {
            'size': 'median'
        }
    )
level0_gr_median = level0_gr_median.rename(columns={'size':'median'})
level0_gr_median['median_mo'] = level0_gr_median['median'].apply(lambda x: azrael.convert_size(x, to_size='mo'))
level0_gr_median = level0_gr_median.reset_index()
level0_gr_median['path1'] = level0_gr_median['path0'] + " - Sous-total"
level0_gr_median['path2'] = np.nan
level0_gr_median['mimetype'] = np.nan
level0_gr_median  = level0_gr_median[['path0', 'path1', 'path2', 'mimetype', 'median', 'median_mo']]

In [32]:
level1_gr_median = az.groupby(['path0', 'path1']).agg(
        {
            'size': 'median'
        }
    )
level1_gr_median = level1_gr_median.rename(columns={'size':'median'})
level1_gr_median['median_mo'] = level1_gr_median['median'].apply(lambda x: azrael.convert_size(x, to_size='mo'))
level1_gr_median = level1_gr_median.reset_index()
level1_gr_median['path2'] = level1_gr_median['path0'] + " | " + level1_gr_median['path1'] + " - Sous-total"
level1_gr_median['mimetype'] = np.nan
level1_gr_median = level1_gr_median[['path0', 'path1', 'path2', 'mimetype', 'median', 'median_mo']]

In [33]:
level2_gr_median = az.groupby(['path0', 'path1', 'path2']).agg(
        {
            'size': 'median'
        }
    )
level2_gr_median = level2_gr_median.rename(columns={'size':'median'})
level2_gr_median['median_mo'] = level2_gr_median['median'].apply(lambda x: azrael.convert_size(x, to_size='mo'))
level2_gr_median = level2_gr_median.reset_index()
level2_gr_median['mimetype'] = level2_gr_median['path0'] + " | " + level2_gr_median['path1'] + " | " + level2_gr_median['path2'] + " - Sous-total"
level2_gr_median = level2_gr_median[['path0', 'path1', 'path2', 'mimetype', 'median', 'median_mo']]

In [34]:
level3_gr_median = az.groupby(['path0', 'path1', 'path2', 'mimetype']).agg(
        {
            'size': 'median'
        }
    )
level3_gr_median = level3_gr_median.rename(columns={'size':'median'})
level3_gr_median['median_mo'] = level3_gr_median['median'].apply(lambda x: azrael.convert_size(x, to_size='mo'))
level3_gr_median = level3_gr_median.reset_index()

In [38]:
main_gr_median = pd.concat([level0_gr_median, level1_gr_median, level2_gr_median, level3_gr_median])

,path0,path1,path2,mimetype,median,median_mo
0,BNR_SAUV,BNR_SAUV - Sous-total,NaN,NaN,26835584.0,25.59
1,BNR_TAMPON,BNR_TAMPON - Sous-total,NaN,NaN,923353.0,0.88
2,BNR_VERIF,BNR_VERIF - Sous-total,NaN,NaN,883527.0,0.84
0,BNR_SAUV,AMR,BNR_SAUV | AMR - Sous-total,NaN,1657019.0,1.58
1,BNR_SAUV,CSV,BNR_SAUV | CSV - Sous-total,NaN,27758972.0,26.47
...,...,...,...,...,...,...
231,BNR_VERIF,PAR,PAR_TRI,image/tiff,265029277.0,252.75
232,BNR_VERIF,PAR,PAR_TRI,video/mp4,50968643.0,48.61
233,BNR_VERIF,PAR,PAR_TRI,video/x-ms-wmv,57098054.0,54.45
234,BNR_VERIF,VERSION TIF 300 DPI,PRA_BDR,image/tiff,17439985.0,16.63


In [104]:
res = pd.merge(gr, gr0,
               on=['path0', 'path1', 'path2', 'mimetype'],
               how='left')
res = res.set_index(['path0', 'path1', 'path2', 'mimetype'])

nb  \
path0     path1                  path2                                        mimetype                                                      
BNR_SAUV  AMR                    AMR_AFF                                      image/tiff                                             1074   
                                                                              BNR_SAUV | AMR | AMR_AFF - Sous-total                  1074   
                                 AMR_CIM                                      image/tiff                                            16070   
                                                                              image/x-xcf                                               1   
                                                                              BNR_SAUV | AMR | AMR_CIM - Sous-total                 16071   
...                                                                                                                                   ...   
BNR_VERIF VERSION TIF 600 DPI    PLS                                          image/tiff                                               15   
                                                                              BNR_VERIF | VERSION TIF 600 DPI | PLS - Sous-total       15   
                                 BNR_VERIF | VERSION TIF 600 DPI - Sous-total                                                          15   
          BNR_VERIF - Sous-total                                                                                                  1774555   
Total                                                                                                                             2290099   

                                                                                                                                            size  \
path0     path1                  path2                                        mimetype                                                             
BNR_SAUV  AMR                    AMR_AFF                                      image/tiff                                            143189496675   
                                                                              BNR_SAUV | AMR | AMR_AFF - Sous-total                 143189496675   
                                 AMR_CIM                                      image/tiff                                            262913255657   
                                                                              image/x-xcf                                               18996782   
                                                                              BNR_SAUV | AMR | AMR_CIM - Sous-total                 262932252439   
...                                                                                                                                          ...   
BNR_VERIF VERSION TIF 600 DPI    PLS                                          image/tiff                                               382261455   
                                                                              BNR_VERIF | VERSION TIF 600 DPI | PLS - Sous-total       382261455   
                                 BNR_VERIF | VERSION TIF 600 DPI - Sous-total                                                          382261455   
          BNR_VERIF - Sous-total                                                                                                   7550379668774   
Total                                                                                                                             19354695511205   

                                                                                                                                      size_mo  \
path0     path1                  path2                                        mimetype                                                          
BNR_SAUV  AMR                    AMR_AFF                                      image/ti

In [106]:
st = "BNR_VERIF | VERSION TIF 600 DPI | PLS - Sous-total"
st[-10:] == 'Sous-total'

True

In [108]:
level3_labels = [l for l in az['mimetype'][az['mimetype'].str[-10:] == 'Sous-total']]

In [109]:
level3_labels

[]

In [110]:
az['mimetype'][az['mimetype'].str[-10:] == 'Sous-total']

Series([], Name: mimetype, dtype: object)

In [115]:
res_df = res.reset_index()
level3_labels = [l[:-13] for l in res_df['mimetype'][res_df['mimetype'].str[-10:] == 'Sous-total']]
level3_labels

['BNR_SAUV | AMR | AMR_AFF',
 'BNR_SAUV | AMR | AMR_CIM',
 'BNR_SAUV | AMR | AMR_Obj',
 'BNR_SAUV | AMR | AMR_PHO',
 'BNR_SAUV | AMR | AMR_POP',
 'BNR_SAUV | AMR | AMR_PR',
 'BNR_SAUV | AMR | PAR_LAB',
 'BNR_SAUV | MDF | MDF_MTX',
 'BNR_SAUV | MED | MED_AFF',
 'BNR_SAUV | MED | MED_CHA',
 'BNR_SAUV | MED | MED_CP',
 'BNR_SAUV | MED | MED_IMA',
 'BNR_SAUV | MED | MED_JOU',
 'BNR_SAUV | MED | MED_LET',
 'BNR_SAUV | MED | MED_MON',
 'BNR_SAUV | MED | MED_MS',
 'BNR_SAUV | MED | MED_PHO',
 'BNR_SAUV | MED | MED_PRA',
 'BNR_SAUV | MED | MED_VDM',
 'BNR_SAUV | MUS | MUS_ARC',
 'BNR_SAUV | MUS | MUS_VAI',
 'BNR_SAUV | OBS | OBS_JOU',
 'BNR_SAUV | PAR | PAR_BOH',
 'BNR_SAUV | PAR | PAR_BOU',
 'BNR_SAUV | PAR | PAR_CAR',
 'BNR_SAUV | PAR | PAR_LAI',
 'BNR_SAUV | PAR | PAR_LES',
 'BNR_SAUV | PAR | PAR_NAJ',
 'BNR_SAUV | PAR | PAR_NIE',
 'BNR_SAUV | PAR | PAR_RAI',
 'BNR_SAUV | PAR | PAR_TUR',
 'BNR_SAUV | TCG | TCG_PRA',
 'BNR_TAMPON | AMR | AMR_Obj',
 'BNR_TAMPON | AMR | AMR_POP',
 'BNR_TAMPON 